In [2]:
import torch
import torchtext.vocab as Vocab

In [4]:
Vocab.pretrained_aliases.keys()

dict_keys(['charngram.100d', 'fasttext.en.300d', 'fasttext.simple.300d', 'glove.42B.300d', 'glove.840B.300d', 'glove.twitter.27B.25d', 'glove.twitter.27B.50d', 'glove.twitter.27B.100d', 'glove.twitter.27B.200d', 'glove.6B.50d', 'glove.6B.100d', 'glove.6B.200d', 'glove.6B.300d'])

In [5]:
[key for key in Vocab.pretrained_aliases.keys() if 'glove' in key]

['glove.42B.300d',
 'glove.840B.300d',
 'glove.twitter.27B.25d',
 'glove.twitter.27B.50d',
 'glove.twitter.27B.100d',
 'glove.twitter.27B.200d',
 'glove.6B.50d',
 'glove.6B.100d',
 'glove.6B.200d',
 'glove.6B.300d']

In [7]:
cache_dir = '/Users/szki/Code/Datasets/glove'
glove = Vocab.GloVe(name='6B', dim=50, cache=cache_dir)

In [9]:
print(len(glove.stoi))

400000


In [13]:
print(glove.stoi['beautiful'], glove.itos[3366])

3366 beautiful


In [15]:
def knn(W, x, k):
    cos = torch.matmul(W, x.view((-1, ))) / (
            (torch.sum(W * W, dim=1)+1e-9).sqrt() * torch.sum(x * x).sqrt())
    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    return topk, [cos[i].item() for i in topk]

In [17]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.vectors,
                    embed.vectors[embed.stoi[query_token]], k+1)
    for i, c in zip(topk[1:], cos[1:]):
        print('cosine sim=%.3f: %s' % (c, (embed.itos[i])))

In [18]:
get_similar_tokens('chip', 3, glove)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


In [19]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = [embed.vectors[embed.stoi[t]]
            for t in [token_a, token_b, token_c]]

    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.vectors, x, 1)
    return embed.itos(topk[0])

In [20]:
get_analogy('man', 'woman', 'son', glove)

TypeError: 'list' object is not callable